In [2]:
from kafka import KafkaProducer
from kafka.errors import KafkaError
import pandas as pd
from binance.client import Client
import datetime
from binance_config import *
import time
from binance import ThreadedWebsocketManager

In [3]:
# Binance client
api_key = API_KEY   
api_secret = SECRET_KEY
client = Client(api_key=api_key, api_secret=api_secret)

In [10]:
# Binance Threaded web socket manager
twm = ThreadedWebsocketManager(api_key=api_key, api_secret=api_secret)

In [6]:
# Kafka client
producer = KafkaProducer(bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'])

In [ ]:
producer = KafkaProducer(bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'], value_serializer=lambda m: json.dumps(m).encode('ascii'))

In [6]:
symbol = 'BNBBTC'

In [8]:
def flatten_json(nested_json):
    """
        Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [14]:
# start is required to initialise its internal loop
twm.start()


def handle_socket_message(msg):
    # print(f"event type: {msg['e']}")

    # producer.send('test', {symbol : msg})
    print(msg)
    print(flatten_json(msg))


twm.start_kline_socket(callback=handle_socket_message, symbol=symbol)

# multiple sockets can be started
# twm.start_depth_socket(callback=handle_socket_message, symbol=symbol)

# or a multiplex socket can be started like this
# see Binance docs for stream names
# streams = ['bnbbtc@miniTicker', 'bnbbtc@bookTicker']
# twm.start_multiplex_socket(callback=handle_socket_message, streams=streams)

twm.join()

message type: depthUpdate
{'e': 'depthUpdate', 'E': 1628239749878, 's': 'BNBBTC', 'U': 2113230735, 'u': 2113230738, 'b': [['0.00819100', '8.22000000'], ['0.00811300', '12.16000000']], 'a': [['0.00820200', '93.37000000']]}


In [4]:
A = {'e': 'kline', 'E': 1628610790272, 's': 'BNBBTC', 'k': {'t': 1628610780000, 'T': 1628610839999, 's': 'BNBBTC', 'i': '1m', 'f': 155943535, 'L': 155943541, 'o': '0.00816900', 'c': '0.00817000', 'h': '0.00817000', 'l': '0.00816800', 'v': '7.56000000', 'n': 7, 'x': False, 'q': '0.06175672', 'V': '0.87000000', 'Q': '0.00710750', 'B': '0'}}

In [9]:
flatten_json(A)

{'e': 'kline',
 'E': 1628610790272,
 's': 'BNBBTC',
 'k_t': 1628610780000,
 'k_T': 1628610839999,
 'k_s': 'BNBBTC',
 'k_i': '1m',
 'k_f': 155943535,
 'k_L': 155943541,
 'k_o': '0.00816900',
 'k_c': '0.00817000',
 'k_h': '0.00817000',
 'k_l': '0.00816800',
 'k_v': '7.56000000',
 'k_n': 7,
 'k_x': False,
 'k_q': '0.06175672',
 'k_V': '0.87000000',
 'k_Q': '0.00710750',
 'k_B': '0'}

In [ ]:
# start is required to initialise its internal loop
twm.start()

def handle_socket_message(msg):
    print(f"message type: {msg['e']}")
    producer.send('test', key=bytearray(symbol, 'utf-8'), value=bytearray(str(msg), 'utf-8'))
    print(msg)
    
twm.start_kline_socket(callback=handle_socket_message, symbol=symbol)

# multiple sockets can be started
#twm.start_depth_socket(callback=handle_socket_message, symbol=symbol)

# or a multiplex socket can be started like this
# see Binance docs for stream names
#streams = ['bnbbtc@miniTicker', 'bnbbtc@bookTicker']
#twm.start_multiplex_socket(callback=handle_socket_message, streams=streams)

twm.join()

In [ ]:
bnbbtc = client.get_symbol_info('BNBBTC')

payload = client.get_symbol_ticker(symbol='BNBBTC')

In [ ]:
producer.send('test', key=bytearray(payload['symbol'], 'utf-8'), value=bytearray(payload['price'], 'utf-8'))

In [17]:
tickers = client.get_all_tickers()

df = pd.DataFrame(tickers)

In [ ]:
# Asynchronous by default
future = producer.send('test', b'raw_bytes')

# Block for 'synchronous' sends
try:
    record_metadata = future.get(timeout=10)
except KafkaError:
    # Decide what to do if produce request failed...
    log.exception()
    pass

# Successful result returns assigned partition and offset
print (record_metadata.topic)
print (record_metadata.partition)
print (record_metadata.offset)

# produce keyed messages to enable hashed partitioning
producer.send('test', key=b'foo', value=b'bar')

# encode objects via msgpack
producer = KafkaProducer(value_serializer=msgpack.dumps)
producer.send('test', {'key': 'value'})

In [ ]:
# produce json messages
producer = KafkaProducer(value_serializer=lambda payload: json.dumps(payload).encode('ascii'))
producer.send('test', {'key': 'value'})

In [ ]:
# produce asynchronously
for _ in range(100):
    producer.send('my-topic', b'msg')

def on_send_success(record_metadata):
    print(record_metadata.topic)
    print(record_metadata.partition)
    print(record_metadata.offset)

def on_send_error(excp):
    log.error('I am an errback', exc_info=excp)
    # handle exception

# produce asynchronously with callbacks
producer.send('my-topic', b'raw_bytes').add_callback(on_send_success).add_errback(on_send_error)

# block until all async messages are sent
producer.flush()

# configure multiple retries
producer = KafkaProducer(retries=5)